In [ ]:
import os
import sqlite3
import math
from typing import Dict, List, Union, Any
import pandas as pd
import numpy as np


In [ ]:
# ===== CONFIG =====
MAGI_DB_PATH = "/projects/klybarge/pcori_ad/magi/magi_db/magi.db"  # read-only
OUT_DIR = "./"
os.makedirs(OUT_DIR, exist_ok=True)
EDGE_TABLE = "magi_counts_top500"   
TARGETS = ["aa_meas_citalopram_rem",]


In [ ]:
## Update 10 23 25
# RIGHT= k (predictor) and LEFT = Y/j
def analyze_causal_sequence_py(
    df_in: Union[str, pd.DataFrame],
    *,
    name_map: Dict[str, str] = None,     # raw_code -> friendly label (applied to BOTH columns)
    events: List[str] = None,            # event names to KEEP (AFTER recoding). If None: auto-detect
    force_outcome: str = None,           # if provided and present, force this to be the FINAL node (Y)
    lambda_min_count: int = 15           # L-threshold for λ: if n_code < L ⇒ λ_{k,j}=0
) -> Dict[str, Any]:
    """
    MAGI (Python): Reference routine with explicit comments.

    ─────────────────────────────────────────────────────────────────────────────
    COLUMN CONVENTION PER ROW:
      Left  column: target_concept_code  (call this X for this row)
      Right column: concept_code         (call this k for this row)

      n_code_target        ≡  k ∧ X
      n_code_no_target     ≡  k ∧ ¬X
      n_target_no_code     ≡  ¬k ∧ X
      n_no_target          ≡  total(¬X)
      n_target             ≡  total(X)
      n_code               ≡  total(k)
      n_code_before_target ≡  count(k before X)
      n_target_before_code ≡  count(X before k)

    ORIENTATION (locked to your spec):
      • Total effect T_{kY}:   read row (target = Y, code = k).
      • Lambda     λ_{k,j}:    read row (target = j, code = k), and compute
                               λ_{k,j} = n_code_target / n_code  with L-threshold on n_code.

    TEMPORAL SCORE for each node Zi:
      Score(Z_i) = Σ_{j≠i} [ C(Z_i≺Z_j) - C(Z_j≺Z_i) + C(Z_i∧¬Z_j) - C(Z_j∧¬Z_i) ]
      Read from row (target=Z_i, code=Z_j):
        - n_code_before_target      → C(Z_j≺Z_i)
        - n_target_before_code      → C(Z_i≺Z_j)
        - n_code_no_target          → C(Z_j∧¬Z_i)
        - n_target_no_code          → C(Z_i∧¬Z_j)

    T_{kY} from row (Y, k):
      a = n_code_target        (k ∧ Y)
      b = n_code_no_target     (k ∧ ¬Y)
      c = n_target_no_code     (¬k ∧ Y)
      d = n_no_target - b      (¬k ∧ ¬Y)   ← computed on the fly (no extra column needed)
      With sample-size–anchored odds:
         odds_k1 = a/b with guards; odds_k0 = c/d with guards; T = odds_k1 / odds_k0

    DIRECT EFFECTS via backward recursion:
      D_{k,Y} = ( T_{k,Y} - Σ_j λ_{k,j} D_{j,Y} ) / ( 1 - Σ_j λ_{k,j} ),
      where j are downstream nodes between k and Y in the temporal order.

    LOGISTIC LINK:
      logit P(Y=1 | Z) = β0 + Σ_k β_k Z_k  with β_k = log D_{k,Y};
      invalid/nonpositive D map to β_k=0.

    RETURNS a dict with:
      sorted_scores, temporal_order, order_used,
      T_val, D_val, lambda_l, coef_df, beta_0, beta, logit_predictors, predict_proba
    """
    # ── 0) Ingest & validate ───────────────────────────────────────────────────
    df = pd.read_csv(df_in) if isinstance(df_in, str) else df_in.copy()

    need_cols = [
        "target_concept_code", "concept_code",
        "n_code_target", "n_code_no_target",
        "n_target", "n_no_target",
        "n_target_no_code",
        "n_code",
        "n_code_before_target", "n_target_before_code",
    ]
    missing = [c for c in need_cols if c not in df.columns]
    if missing:
        raise ValueError(f"Missing required columns: {', '.join(missing)}")

    # Always treat endpoints as strings to avoid silent filter drop
    df["target_concept_code"] = df["target_concept_code"].astype(str)

    # Optional recoding
    if name_map:
        df["target_concept_code"] = df["target_concept_code"].replace(name_map)
        df["concept_code"]        = df["concept_code"].replace(name_map)

    # Limit to selected events (union if auto)
    if events is None:
        ev_t = df["target_concept_code"].unique().tolist()
        ev_c = df["concept_code"].unique().tolist()
        events = sorted(set(ev_t) | set(ev_c))
    else:
        events = [str(e) for e in events]

    if len(events) < 2:
        raise ValueError("Need at least two events.")

    df = df[df["target_concept_code"].isin(events) & df["concept_code"].isin(events)].copy()

    # Coerce numerics
    num_cols = [
        "n_code_target", "n_code_no_target",
        "n_target", "n_no_target", "n_target_no_code",
        "n_code", "n_code_before_target", "n_target_before_code",
    ]
    for c in num_cols:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0.0)

    # ── 1) Temporal score (read rows: target=Z_i, code=Z_j) ────────────────────
    scores: Dict[str, float] = {}
    for zi in events:
        s = 0.0
        for zj in (x for x in events if x != zi):
            # Row oriented as (Zi, Zj)
            pair = df[(df["target_concept_code"] == zi) & (df["concept_code"] == zj)]
            if pair.empty:
                continue
            c_i_before_j = float(pair["n_target_before_code"].sum(skipna=True))   # Zi before Zj
            c_j_before_i = float(pair["n_code_before_target"].sum(skipna=True))   # Zj before Zi
            c_i_and_not_j = float(pair["n_target_no_code"].sum(skipna=True))      # Zi ∧ ¬Zj
            c_j_and_not_i = float(pair["n_code_no_target"].sum(skipna=True))      # Zj ∧ ¬Zi
            s += (c_i_before_j - c_j_before_i + c_i_and_not_j - c_j_and_not_i)
        scores[zi] = s

    sorted_scores = pd.Series(scores).sort_values(ascending=False)

    # Choose outcome Y: either forced or top-scoring node
    if force_outcome and (force_outcome in sorted_scores.index):
        outcome = force_outcome
        temporal_order = [ev for ev in sorted_scores.index if ev != outcome] + [outcome]
    else:
        outcome = sorted_scores.index[0]
        temporal_order = [ev for ev in sorted_scores.index if ev != outcome] + [outcome]

    events_order = temporal_order              # earliest … → Y
    nodes = events_order[:-1]            # everything before Y

    # ── 2) T and λ (row orientations locked) ───────────────────────────────────
    T_val = pd.Series(0.0, index=nodes, dtype=float)
    D_val = pd.Series(np.nan, index=nodes, dtype=float)
    lambda_l: Dict[str, pd.Series] = {}

    for k in nodes:
        # ---- T_{kY} from the single row (target=Y, code=k) ----
        row_Yk = df[(df["target_concept_code"] == outcome) & (df["concept_code"] == k)]

        # 2×2 cells (a,b,c,d) at (Y,k):
        a = float(row_Yk["n_code_target"].sum(skipna=True))        # k ∧ Y
        b = float(row_Yk["n_code_no_target"].sum(skipna=True))     # k ∧ ¬Y
        c = float(row_Yk["n_target_no_code"].sum(skipna=True))     # ¬k ∧ Y
        # d is not in data; compute from the same row: d = total(¬Y) − (k ∧ ¬Y)
        n_noY = float(row_Yk["n_no_target"].max(skipna=True)) if not row_Yk.empty else 0.0
        d = max(n_noY - b, 0.0)                                    # ¬k ∧ ¬Y

        # Stratum sizes
        N1 = a + b                   # k = 1
        N0 = c + d                   # k = 0

        # Sample-size–anchored odds for k=1 and k=0
        if N1 == 0:
            odds_k1 = 1.0
        else:
            if a == 0:
                odds_k1 = 1.0 / (N1 + 1.0)
            elif b == 0:
                odds_k1 = (N1 + 1.0)
            else:
                odds_k1 = a / b

        if N0 == 0:
            odds_k0 = 1.0
        else:
            if c == 0:
                odds_k0 = 1.0 / (N0 + 1.0)
            elif d == 0:
                odds_k0 = (N0 + 1.0)
            else:
                odds_k0 = c / d

        T_val.loc[k] = float(odds_k1 / odds_k0) if odds_k0 > 0 else (N1 + 1.0)

        # ---- λ_{k,j} from rows (target=j, code=k): λ = n_code_target / n_code ----
        pos_k = events_order.index(k)
        js = events_order[pos_k + 1 : -1] if pos_k < len(events_order) - 1 else []

        lam_pairs = []
        for j in js:
            row_jk = df[(df["target_concept_code"] == j) & (df["concept_code"] == k)]
            if row_jk.empty:
                lam_pairs.append((j, 0.0))
                continue

            num = float(pd.to_numeric(row_jk["n_code_target"], errors="coerce").fillna(0.0).sum())
            den = float(pd.to_numeric(row_jk["n_code"],        errors="coerce").fillna(0.0).sum())

            # L-threshold on the conditioning size n_code (count of k)
            if (den <= 0) or (den < lambda_min_count):
                lam_pairs.append((j, 0.0))
                continue

            lam = num / den
            lam = 0.0 if not np.isfinite(lam) else float(min(max(lam, 0.0), 1.0))
            lam_pairs.append((j, lam))

        lambda_l[k] = pd.Series({j: v for j, v in lam_pairs}, dtype=float)

    # ── 3) Backward recursion for direct effects D ─────────────────────────────
    # Last antecedent (just before Y): no downstream → D = T
    if len(nodes) >= 1:
        last_anc = nodes[-1]
        D_val.loc[last_anc] = T_val.loc[last_anc]

    # Walk backward for the rest
    if len(nodes) > 1:
        for k in list(reversed(nodes[:-1])):
            lam_vec = lambda_l.get(k, pd.Series(dtype=float))
            downstream = list(lam_vec.index)  # j nodes after k (already resolved)
            lam_vals = lam_vec.reindex(downstream).fillna(0.0).to_numpy()
            D_down  = pd.to_numeric(D_val.reindex(downstream), errors="coerce").fillna(0.0).to_numpy()

            num = T_val.loc[k] - float(np.nansum(lam_vals * D_down))
            den = 1.0 - float(np.nansum(lam_vals))

            if (not np.isfinite(den)) or den == 0.0:
                D_val.loc[k] = T_val.loc[k]            # neutralize if pathological
            else:
                tmp = num / den
                D_val.loc[k] = tmp if np.isfinite(tmp) else T_val.loc[k]

    # ── 4) Logistic link (β) and predict_proba ─────────────────────────────────
    # Intercept β0 from marginal prevalence of Y (rows with target == Y)
    resp_rows = df[df["target_concept_code"] == outcome]
    n_t = float(resp_rows["n_target"].max()) if not resp_rows.empty else np.nan
    n_n = float(resp_rows["n_no_target"].max()) if not resp_rows.empty else np.nan
    denom = n_t + n_n
    p_y = 0.5 if (not np.isfinite(denom) or denom <= 0) else (n_t / denom)
    p_y = min(max(p_y, 1e-12), 1 - 1e-12)
    beta_0 = float(np.log(p_y / (1 - p_y)))

    # β_k = log D_{k,Y}; invalid/nonpositive → 0
    D_clean = pd.to_numeric(D_val, errors="coerce").astype(float)
    beta_vals = np.log(D_clean.where(D_clean > 0.0)).replace([np.inf, -np.inf], np.nan).fillna(0.0)

    coef_df = pd.DataFrame({
        "predictor": list(beta_vals.index) + ["(intercept)"],
        "beta":      list(beta_vals.values) + [beta_0],
    })

    # Vectorized predict_proba
    predictors = list(beta_vals.index)
    beta_vec = beta_vals.values

    def predict_proba(Z: Union[Dict[str, Any], pd.Series, np.ndarray, List[float], pd.DataFrame]):
        """
        Compute P(Y=1|Z) using: logit P = β0 + Σ_k β_k Z_k.
        Z can be:
          - dict/Series mapping predictor name -> 0/1
          - 1D/2D numpy/list with columns ordered as `predictors`
          - DataFrame containing any/all of `predictors` (others ignored)
        """
        def sigmoid(x):
            x = np.clip(x, -700, 700)  # numerical stability for large |η|
            return 1.0 / (1.0 + np.exp(-x))

        if isinstance(Z, pd.DataFrame):
            M = Z.reindex(columns=predictors, fill_value=0.0).astype(float).to_numpy()
            return sigmoid(beta_0 + M @ beta_vec)

        if isinstance(Z, (dict, pd.Series)):
            v = np.array([float(Z.get(p, 0.0)) for p in predictors], dtype=float)
            return float(sigmoid(beta_0 + float(v @ beta_vec)))

        arr = np.asarray(Z, dtype=float)
        if arr.ndim == 1:
            if arr.size != len(predictors):
                raise ValueError(f"Expected {len(predictors)} features in order: {predictors}")
            return float(sigmoid(beta_0 + float(arr @ beta_vec)))
        if arr.ndim == 2:
            if arr.shape[1] != len(predictors):
                raise ValueError(f"Expected shape (*,{len(predictors)}), got {arr.shape}")
            return sigmoid(beta_0 + arr @ beta_vec)

        raise ValueError("Unsupported input for predict_proba")

    # ── 5) Package results ─────────────────────────────────────────────────────
    return {
        "sorted_scores": sorted_scores,
        "temporal_order": temporal_order,
        "order_used": events_order,
        "T_val": T_val,
        "D_val": D_val,
        "lambda_l": lambda_l,
        "coef_df": coef_df,
        "beta_0": beta_0,
        "beta": pd.Series(beta_vec, index=predictors, dtype=float),
        "logit_predictors": predictors,
        "predict_proba": predict_proba,
    }
